### Tips from prof

- Narrow scope of work (e.g. court level)

- Could try both binary/multi-class model outcomes and compare the performance 

- Change user from layperson to legal professional (and mention that this project is a stepping stone towards having layperson use the model)

- Link features to predicted outcome (if time permits can try using XGBoost with LIME for model interpretability)

- Can also try to see accuracy of models with different areas of law, lowest accuracy may be hardest area of law to predict


### Data setup

In [6]:
import os
import re
import spacy 
from spacy import displacy
import json
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel, LsiModel, HdpModel
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
import ast

import pyLDAvis
import pyLDAvis.gensim_models

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Zhiyi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Zhiyi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
# Load CSV files into DataFrames
areas_of_law_df = pd.read_csv("data/prediction_data/areas_of_law.csv")
coram_df = pd.read_csv("data/prediction_data/coram.csv")
fact_themes_df = pd.read_csv("data/prediction_data/fact_themes.csv")
sg_legal_cases_df = pd.read_csv("data/prediction_data/sg_legal_cases_dataset.csv")
target_rulings_df = pd.read_csv("data/prediction_data/target_rulings.csv")
# Load the JSON file into a dictionary
with open('data/prediction_data/issues.json') as f:
    issues_data = [json.loads(line) for line in f]
issues_df = pd.DataFrame(issues_data)

# Load the JSON file into a dictionary
with open('data/rawish_data/facts.json') as f:
    facts_data = [json.loads(line) for line in f]
raw_facts_df = pd.DataFrame(facts_data)

# Merge DataFrames
merged_df = pd.merge(areas_of_law_df, coram_df, on='casename', how='outer')
merged_df = pd.merge(merged_df, fact_themes_df, on='casename', how='outer')
merged_df = pd.merge(merged_df, sg_legal_cases_df, left_on='casename', right_on='filename', how='outer')
# merged_df = pd.merge(merged_df, issues_df, on='casename', how='outer')
merged_df = pd.merge(merged_df, target_rulings_df, on='casename', how='outer')

merged_df.drop(columns=['Unnamed: 0'], inplace=True)
merged_df.drop(columns=['filename'], inplace=True)

# Display the resulting DataFrame
print(merged_df.head())

           casename                                        area_of_law  \
0   2000_SGCA_1.pdf  {'civil procedure': ['pleadings'], 'res judica...   
1  2000_SGCA_10.pdf  {'contract': ['formation'], 'equity': ['defenc...   
2  2000_SGCA_11.pdf  {'contract': ['discharge'], 'damages': ['asses...   
3  2000_SGCA_12.pdf  {'courts and jurisdiction': ['court of appeal'...   
4  2000_SGCA_13.pdf                     {'criminal law': ['offences']}   

                                           Coram  themes court_level  \
0  ['Chao Hick Tin , L P Thean , Yong Pung How']     5.0        SGCA   
1   ['Chao Hick Tin , Tan Lee Meng , L P Thean']     5.0        SGCA   
2   ['Chao Hick Tin , Tan Lee Meng , L P Thean']     5.0        SGCA   
3   ['Chao Hick Tin , Tan Lee Meng , L P Thean']     5.0        SGCA   
4   ['Chao Hick Tin , Lai Kew Chai , L P Thean']    12.0        SGCA   

         target  
0    Favourable  
1    Favourable  
2    No outcome  
3  Unfavourable  
4  Unfavourable  


### Data Preprocessing

In [8]:
nan_counts = merged_df.isna().sum()
print(nan_counts)

#nas are probably those reassigned cases, coram has 7, i just drop them for now
na_target_rows = merged_df[merged_df['target'].isna()]
print(na_target_rows)

merged_df.dropna(axis=0, inplace=True)
print(merged_df.isna().sum())

#remove empty lists
merged_df = merged_df.query("area_of_law != '[]'")

#target is unbalanced
target_counts = merged_df['target'].value_counts()
print(target_counts)

casename        0
area_of_law     0
Coram           7
themes         47
court_level     0
target         47
dtype: int64
              casename area_of_law  \
241  2000_SGHC_257.pdf          []   
274  2000_SGHC_290.pdf          []   
412   2001_SGCA_66.pdf          []   
432  2001_SGHC_101.pdf          []   
438  2001_SGHC_108.pdf          []   
442  2001_SGHC_111.pdf          []   
448  2001_SGHC_118.pdf          []   
457  2001_SGHC_128.pdf          []   
460  2001_SGHC_130.pdf          []   
462  2001_SGHC_132.pdf          []   
475  2001_SGHC_148.pdf          []   
478  2001_SGHC_150.pdf          []   
479  2001_SGHC_151.pdf          []   
489  2001_SGHC_163.pdf          []   
498  2001_SGHC_174.pdf          []   
536  2001_SGHC_214.pdf          []   
537  2001_SGHC_215.pdf          []   
544  2001_SGHC_222.pdf          []   
546  2001_SGHC_224.pdf          []   
550  2001_SGHC_228.pdf          []   
551  2001_SGHC_229.pdf          []   
555  2001_SGHC_232.pdf          []   
564  

In [9]:
merged_df['area_of_law'] = merged_df['area_of_law'].apply(ast.literal_eval)
merged_df['Coram'] = merged_df['Coram'].apply(ast.literal_eval)
merged_df.head(3)

,casename,area_of_law,Coram,themes,court_level,target
0,2000_SGCA_1.pdf,"{'civil procedure': ['pleadings'], 'res judica...","[Chao Hick Tin , L P Thean , Yong Pung How]",5.0,SGCA,Favourable
1,2000_SGCA_10.pdf,"{'contract': ['formation'], 'equity': ['defenc...","[Chao Hick Tin , Tan Lee Meng , L P Thean]",5.0,SGCA,Favourable
2,2000_SGCA_11.pdf,"{'contract': ['discharge'], 'damages': ['asses...","[Chao Hick Tin , Tan Lee Meng , L P Thean]",5.0,SGCA,No outcome


### Flatten areas_of_law

In [61]:
all_areas = []

for index, row in merged_df.iterrows():

    areas = row['area_of_law']
    flat_areas = []
    for main_area, sub_areas in areas.items():
        flat_areas.append(main_area)
        flat_areas.extend(sub_areas)
    all_areas.append(flat_areas)

### One hot encoding

In [69]:
# one-hot encode aol
mlb = MultiLabelBinarizer()
binary_features = mlb.fit_transform(all_areas)

binary_aol_df = pd.DataFrame(binary_features, columns=mlb.classes_)

processed_df = pd.concat([merged_df.drop('area_of_law', axis=1), binary_aol_df], axis=1)

print(processed_df.head(3))

           casename                                        Coram  themes  \
0   2000_SGCA_1.pdf  [Chao Hick Tin , L P Thean , Yong Pung How]     5.0   
1  2000_SGCA_10.pdf   [Chao Hick Tin , Tan Lee Meng , L P Thean]     5.0   
2  2000_SGCA_11.pdf   [Chao Hick Tin , Tan Lee Meng , L P Thean]     5.0   

  court_level      target  "a larger sum being repaid"  "abet"  \
0        SGCA  Favourable                          0.0     0.0   
1        SGCA  Favourable                          0.0     0.0   
2        SGCA  No outcome                          0.0     0.0   

   "an interest in any matter"  "any action proposed or contemplated"  \
0                          0.0                                    0.0   
1                          0.0                                    0.0   
2                          0.0                                    0.0   

   "any person"  ...  writ of  seizure and sale  writ of summons  \
0           0.0  ...                        0.0              0.0   
1

In [70]:
processed_df = processed_df[processed_df['Coram'].apply(lambda x: isinstance(x, list))]

In [71]:
# one-hot encode coram
mlb = MultiLabelBinarizer()
binary_features = mlb.fit_transform(processed_df['Coram'])

binary_coram_df = pd.DataFrame(binary_features, columns=mlb.classes_)

processed_df = pd.concat([processed_df.drop('Coram', axis=1), binary_coram_df], axis=1)

print(processed_df.head())

           casename  themes court_level        target  \
0   2000_SGCA_1.pdf     5.0        SGCA    Favourable   
1  2000_SGCA_10.pdf     5.0        SGCA    Favourable   
2  2000_SGCA_11.pdf     5.0        SGCA    No outcome   
3  2000_SGCA_12.pdf     5.0        SGCA  Unfavourable   
4  2000_SGCA_13.pdf    12.0        SGCA  Unfavourable   

   "a larger sum being repaid"  "abet"  "an interest in any matter"  \
0                          0.0     0.0                          0.0   
1                          0.0     0.0                          0.0   
2                          0.0     0.0                          0.0   
3                          0.0     0.0                          0.0   
4                          0.0     0.0                          0.0   

   "any action proposed or contemplated"  "any person"  "appeal"  ...  \
0                                    0.0           0.0       0.0  ...   
1                                    0.0           0.0       0.0  ...   
2          

#### Helper functions for Topic Modelling 

In [11]:
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[0-9]', '', text)
    text = re.sub(r'\W*\b(?!no)\w{1,2}\b', '', text)
    stop_words = set(stopwords.words('english'))
    legal_stopwords = ('appellant', 'respondent', 'plaintiff', 'defendant', 'mr', 'mrs', 'dr', 'mdm', 'court','version', 'hr', 'would', 'case', 'sghc', 'court', 'sgca', 'slr', 'sgdc', 'also', 'first', 'person', 'statement', 'line', 'para', 'fact', 'one', 'may', 'time', 'could', 'next')
    stop_words.update(legal_stopwords)
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    return words

### Topic Modelling For Facts

In [14]:
data = []

raw_facts_df = raw_facts_df[['casename', 'facts']]    
raw_facts_df['processed_facts'] = raw_facts_df['facts'].apply(preprocess_text)
raw_facts_df.drop(columns=['facts'], inplace=True)
print(raw_facts_df["processed_facts"])
print("Finished preprocessing text")

texts, article = [], []
print("Performing topic modelling")
for fact in raw_facts_df['processed_facts']:
    for word in fact:
        article.append(word)
        
    texts.append(article)
    article = []

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
lda_model = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)
print(lda_model.show_topics())
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

0       [fact, widow, tan, geok, tee, deceased, sue, c...
1       [fact, surrounding, circumstance, including, a...
2       [background, appellant, french, company, secon...
3       [background, microsoft, adobe, autodesk, compa...
4       [fact, mere, assertion, suffice, exh, said, st...
                              ...                        
8515    [fact, accused, low, sze, song, low, year, old...
8516    [fact, giving, opinion, representation, amount...
8517    [fact, party, karan, bagga, litigant, proceedi...
8518                                                   []
8519    [fact, party, towa, company, incorporated, jap...
Name: processed_facts, Length: 8520, dtype: object
Finished preprocessing text
Performing topic modelling
[(0, '0.012*"ltd" + 0.012*"contract" + 0.011*"pte" + 0.008*"work" + 0.006*"hr" + 0.006*"project" + 0.006*"payment" + 0.006*"party" + 0.005*"claim" + 0.005*"dated"'), (1, '0.013*"company" + 0.012*"ltd" + 0.011*"singapore" + 0.009*"pte" + 0.008*"affidavit" +

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.184379  0.060536       1        1  18.720352
6      0.031525  0.016329       2        1  14.583921
9      0.072670  0.033042       3        1  13.654031
0      0.063267 -0.041980       4        1  12.358811
5      0.080282  0.008588       5        1  11.792996
1      0.068037 -0.007891       6        1   8.979148
8     -0.114844 -0.100734       7        1   6.994411
2     -0.017343 -0.010533       8        1   5.966529
7     -0.023868  0.042325       9        1   3.709567
4      0.024653  0.000319      10        1   3.240233, topic_info=          Term          Freq         Total Category  logprob  loglift
815    accused  26996.000000  26996.000000  Default  30.0000  30.0000
44       share  18734.000000  18734.000000  Default  29.0000  29.0000
3283    victim  10326.000000  10326.000000  Default  28.0000  28.0000
1742  property  15398.000000  15398.000000  Default  27.0000  27.0000
108    company  22187.000000  22187.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
132      dated    488.484204  15304.188377  Topic10  -5.7977  -0.0150
453        two    474.350707  15335.789175  Topic10  -5.8270  -0.0465
589        ltd    480.346289  25098.671992  Topic10  -5.8145  -0.5265
63      agreed    461.340304   8480.453865  Topic10  -5.8548   0.5181
826     charge    462.095018  14469.127127  Topic10  -5.8532  -0.0145

[872 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
25122      1  0.978795    abang
25122      8  0.010358    abang
21366      8  0.962785  abdulla
5304       3  0.006400      abt
5304       4  0.012800      abt
...      ...       ...      ...
2468       6  0.046589     yong
2468       7  0.048706     yong
2468       8  0.232096     yong
2468       9  0.040236     yong
2468      10  0.031765     yong

[4492 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 7, 10, 1, 6, 2, 9, 3, 8, 5])

### Topic Modelling For Issues

In [16]:
data = []

issues_df = issues_df[['casename', 'issues']]    
issues_df['processed_issues'] = issues_df['issues'].apply(preprocess_text)
issues_df.drop(columns=['issue'], inplace=True)
print(issues_df["processed_issues"])
print("Finished preprocessing text")

texts, article = [], []
print("Performing topic modelling")
for issue in issues_df['processed_issues']:
    for word in issue:
        article.append(word)
        
    texts.append(article)
    article = []

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
lda_model = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)
print(lda_model.show_topics())
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

KeyboardInterrupt: 

#### splitting

In [42]:
X = merged_df.drop(columns=['target'])
y = merged_df['target']

stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, remaining_index in stratified_split.split(X, y):
    X_train, X_test_val = X.iloc[train_index], X.iloc[remaining_index]
    y_train, y_test_val = y.iloc[train_index], y.iloc[remaining_index]

#balanced dataset (target variable was imbalanced Favourable 5006 Unfavourable 2523 No outcome 984)
#randomly found one online, can be changed -> need to check am i doing this right 
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

#split further from X_test_val into X_val and X_test
X_val, X_test, y_val, y_test = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42, stratify=y_test_val)

### Feature Engineering

In [ ]:
## One hot encoding
## vector embedding
## pipeline

### Modeling

In [ ]:
# Perform modelling

### Evaluation

In [ ]:
## Perform Evaluation